In [137]:
import pandas as pd

import matplotlib.pyplot as plt

import matplotlib as mpl

import numpy as np

%matplotlib inline

plt.style.use('ggplot')

pd.set_option('precision',3)

from IPython.display import display, HTML

pd.__version__

u'0.17.1'

In [138]:
SSF_subsidies = pd.read_csv("SSF_subsidies.csv")
countries = pd.read_csv("Country_Data.csv")
catch = pd.read_csv("SAU_Catches.csv")
type_EEZ = pd.read_csv("Subtype_Fleet_EEZ.csv")
all_subsidies = pd.read_csv("Subsidies2009.csv")

In [ ]:
print SSF_subsidies.columns.values
countries_assessed = SSF_subsidies.groupby(['RegionName','Countries']).sum()
countries_assessed = countries_assessed.loc[:,['total_subsidies']]
countries_assessed.to_csv('countries_assessed.csv')

['ID' 'Cnumber' 'RegionID' 'RegionName' 'Type' 'SubType' 'Category'
 'total_subsidies' 'NewData' 'Developed' 'HDI_2005' 'Countries'
 'SSFsubsidies_assessed_percent' 'weighted_by_sectorsize' 'Data_group'
 'SSF_totalCatch_EEZ_percent' '%SSF_totalCatch_Country' 'SSF_USD_1000'
 'Comment' 'DocSource' 'SourceVisitDate' 'Ref' 'URL' 'User' 'OldComment']


In [ ]:
catch.groupby(['EEZ']).sum()

In [ ]:
type_EEZ

In [ ]:

print SSF_subsidies.columns.values
Appendix1 = SSF_subsidies.loc[:,['Countries','SubType','SSF_USD_1000','Data_group']]
Appendix1.to_csv('Appendix1.csv')

In [ ]:


SSF_subsidies['SSF_subsidies_percent'] = SSF_subsidies['SSF_USD_1000']/SSF_subsidies['total_subsidies']


#  merge original all subsidies data with to align with subsidy 
#  type and whether the EEZ catch data or the FLeet catch data apply, 
#  see type_EEZ data, this is in preparation to later merge with SAU_catch data 

subsidies_type = pd.merge(
    left=all_subsidies,
    right=type_EEZ,
    how='left'
)


#  now this needs to be merged with country data to have coordinates per country
#  and have each country data point assigned to a subregion (thanks to Andres data)

subsidies_type_countries = pd.merge(
    left=subsidies_type,
    right=countries,
    how='left'
)


#  calculate SSF catch percent by dividing SSF catch / total catch

catch['SSF_catch_percent']=catch['SSF_catch']/catch['total_catch']


#  merge with SSF catch from SAU_catch data

subsidies_type_countries_catch = pd.merge(
    left=subsidies_type_countries,
    right=catch, how='left'
)

#  calculating mean SSF catch per subregion, 
#  needed to use to fill gaps for SSF subsidies that have not been assessed

catch_countries = pd.merge(
    left=catch, 
    right=countries, 
    how='left'
)

In [ ]:

catch_by_subregion = catch_countries.groupby(['Subregion']).sum()
catch_by_subregion['SSF_subregion_catch'] = (
    catch_by_subregion['SSF_catch']/catch_by_subregion['total_catch']
)

catch_by_subregion = catch_by_subregion.reset_index()

SSF_subregion_catch = catch_by_subregion.loc[:,['Subregion','SSF_subregion_catch']]



#  merging mean catch with all subsidies data
subsidies_type_countries_catch_meancatch = pd.merge(
    left=subsidies_type_countries_catch,right=SSF_subregion_catch, how='left'
) 

#  preparing SSF subsidies by merging with country data
SSF_subsidies_countries = pd.merge(
    left=SSF_subsidies,right=countries, how='left')

#  from SSF subsidies assessed calculate mean per subregion and per type 
#  to fill gaps of countries that have not been assessed
SSF_by_subregion = SSF_subsidies_countries.groupby(
    ['Subregion','Type']).sum(
)


SSF_by_subregion['SSF_subsidies_subregion'] = (
    SSF_by_subregion['SSF_USD_1000']/SSF_by_subregion['total_subsidies']
)
SSF_by_subregion = SSF_by_subregion.reset_index()
SSF_subsidies_subregion = SSF_by_subregion.loc[:,['Subregion','Type','SSF_subsidies_subregion']]


In [ ]:

#  as not all subsidies types are covered by countries assessed, 
#  mean values for region are being calculated to replace the empty cells
SSF_by_region = SSF_subsidies_countries.groupby(
    ['RegionName','Type']).sum(
)
SSF_by_region['SSF_subsidies_region'] = (
    SSF_by_region['SSF_USD_1000']/SSF_by_region['total_subsidies']
)
SSF_by_region = SSF_by_region.reset_index()
SSF_subsidies_region = SSF_by_region.loc[
    :,['RegionName','Type','SSF_subsidies_region']
]

#  merge mean subsidies by type and subregion to all subsidies data
catch_subsidies_subregion = pd.merge(
    left=subsidies_type_countries_catch_meancatch,right=SSF_subsidies_subregion, how='left'
)


#  merge mean subsidies by type and region to all subsidies data that now have included total subsidies per subregion
catch_subsidies_subregion_region = pd.merge(
    left=catch_subsidies_subregion,right=SSF_subsidies_region, how='left'
)

#  fill gaps from mean subsidies per subregion with mean subsidies per region and name new column ['subsidies_subregion_all']

#  create new column and name ['SSF_subsidies_true] and state true for all values > 0 

catch_subsidies_subregion_region['total_subsidies_true'] = np.where(
    catch_subsidies_subregion_region['total_subsidies'] > 0, True, False
)
    
def check_data(index=0):
    df = catch_subsidies_subregion_region
    print("ssf all: {}, total: {}".format(df['total_subsidies_true'][index], df['total_subsidies'][index]))
    
check_data(1946)

#  make a new column that states True for NaN in subsidies per subregion and name ['total_subregion_NaN']

catch_subsidies_subregion_region['total_subregion_NaN'] = pd.isnull(catch_subsidies_subregion_region['SSF_subsidies_subregion'])


def check_data(index=0):
    print("ssf all: {}, total: {}".format(catch_subsidies_subregion_region['SSF_subsidies_subregion'][index], catch_subsidies_subregion_region['total_subregion_NaN'][index]))
    
check_data(1946)



In [ ]:
#  create a new column that states True when ['total_subregion_NaN'] is True and ['SSF_subsidies_true] is True
#  name new column ['total_subregion_all']
#  this shows when True that there is a total subsidy for that category 
#  but no value from total subregion subsidy data and therefore we will replace with the total region value

catch_subsidies_subregion_region['total_subregion_all'] = np.where((catch_subsidies_subregion_region['total_subsidies_true'] == True) & (catch_subsidies_subregion_region['total_subregion_NaN'] == True), True, False)


def check_data(index=0):
    print("ssf all: {0}, total: {1}, isNaN: {2}".format(catch_subsidies_subregion_region['total_subregion_all'][index],catch_subsidies_subregion_region['total_subsidies_true'][index], catch_subsidies_subregion_region['total_subregion_NaN'][index]))

check_data(1056)

In [ ]:
#  create new column and name ['subsidies_subregion_all'] to use value from subregion or from region
#  when ['total_subregion_all'] is true use region data when false use subregion data


#  catch_subsidies_subregion_region['subsidies_subregion_all'] = 
#  np.where(catch_subsidies_subregion_region['total_subregion_all'] == True, 
#  catch_subsidies_subregion_region['SSF_subsidies_region'], 
#  catch_subsidies_subregion_region['SSF_subsidies_subregion'])


catch_subsidies_subregion_region['subsidies_subregion_all'] = np.where(catch_subsidies_subregion_region['total_subregion_all'] == True, catch_subsidies_subregion_region['SSF_subsidies_region'], catch_subsidies_subregion_region['SSF_subsidies_subregion'])

catch_subsidies_subregion_region

def check_data(index=0):
    print("ssf all: {0}, total: {1}, isNaN: {2}".format(catch_subsidies_subregion_region['subsidies_subregion_all'][index],catch_subsidies_subregion_region['total_subregion_all'][index], catch_subsidies_subregion_region['SSF_subsidies_region'][index]))

check_data(1146)

In [ ]:
np.sum(catch_subsidies_subregion_region['total_subsidies'])

In [ ]:
countries.columns.values

In [ ]:
#  to shorten the create dataframe, creat subsidies1 that only include at this point relevant columns

subsidies1 = catch_subsidies_subregion_region.loc[:,['RegionName','Subregion','Countries','CLon', 'CLat','SIDS',
       'TotPop', 'HDI', 'Developed', 'GDP', 'EU member', 'Category','SubType','Type','EEZ','total_catch', 'total_subsidies','SSF_catch_percent','SSF_subregion_catch','subsidies_subregion_all']]

#  create a new column 'Factor' and calculate as adjustment value for total subsidies per subregion and total SSF catch per subregion 
#  used to fill gaps for country that have not been assessed, this is done per subsidy subtype also

subsidies1['Factor'] = subsidies1['SSF_catch_percent']/subsidies1['SSF_subregion_catch']

#  multiply the "Factor" value with the mean subsidy value per subregion
#  to estimate the value for SSF subsidies per subsidy type for the countries that have not been assessed
subsidies1['SSF_subsidies_estimates'] = subsidies1['Factor']*subsidies1['subsidies_subregion_all']


#  to make sure none of the estimated values exceed 1 (100%) the SSF_subsidies_estimate will be capped at 1

subsidies1['SSF_subsidies_estimates_big'] = subsidies1['SSF_subsidies_estimates'] > 1

subsidies1['SSF_estimates_corrected'] = np.where(subsidies1['SSF_subsidies_estimates_big'] == True, 1, subsidies1['SSF_subsidies_estimates'])

def check_data(index=0):
    print("ssf_estimates: {0}, ssf_big: {1}, SSF_corrected: {2}".format(subsidies1['SSF_subsidies_estimates'][index],subsidies1['SSF_subsidies_estimates_big'][index], subsidies1['SSF_estimates_corrected'][index]))

check_data(1002)


In [ ]:
#  shorten SSF subsidy data, that contains originally assessed SSF subsidy values

SSF_subsidies_short = SSF_subsidies_countries.loc[:,[
        'Countries','Type','SSFsubsidies_assessed_percent','weighted_by_sectorsize',
        'Data_group','SSF_USD_1000','SSF_subsidies_percent']
                                                 ]


#  merge estimated values for subsidies with SSF assessed subsidies data
SSF_subsidies_estimates_assessed = pd.merge(
    left=subsidies1,right=SSF_subsidies_short,how='left'
)

#  use fillna function to use estimated values for SSF subsidies to fill gaps of countries that have not been assessed
SSF_subsidies_estimates_assessed['SSF_subsidies_percent_all'] = SSF_subsidies_estimates_assessed[
    'SSF_subsidies_percent'].fillna(
    SSF_subsidies_estimates_assessed['SSF_estimates_corrected']
)

#  calculate USD values for all SSF for all subtypes and countries form the estimated and assessed percentages
SSF_subsidies_estimates_assessed['SSF_subsidies_USD'] = (
    SSF_subsidies_estimates_assessed['total_subsidies']
)*SSF_subsidies_estimates_assessed['SSF_subsidies_percent_all']

#  rename column names to make it easy for visualization and graphs and calculate LSF subsidies

SSF_subsidies_estimates_assessed = SSF_subsidies_estimates_assessed.rename(columns={'SSF_subsidies_USD':'SSF subsidies'})

SSF_subsidies_estimates_assessed['LSF subsidies'] = SSF_subsidies_estimates_assessed['total_subsidies'] - SSF_subsidies_estimates_assessed['SSF subsidies']

SSF_subsidies_estimates_assessed.to_csv('SSF_subsidies_all.csv')

print np.sum(SSF_subsidies_estimates_assessed['SSF subsidies'])
print np.sum(SSF_subsidies_estimates_assessed['total_subsidies'])
print np.sum(SSF_subsidies_estimates_assessed['LSF subsidies'])

subsidies1.to_csv('check_subsidies1.csv')


In [ ]:

SSFglobalpercent = np.sum(SSF_subsidies_estimates_assessed['SSF subsidies'])/np.sum(catch_subsidies_subregion_region['total_subsidies'])*100

SSFglobalpercent

In [ ]:

#  prepare for plotting global subsidies per category

Subsidies_per_Category = SSF_subsidies_estimates_assessed.groupby(['Category']).sum()

Subsidies_per_Category['SSF percent'] = Subsidies_per_Category['SSF subsidies']/Subsidies_per_Category['total_subsidies']*100
print Subsidies_per_Category['SSF percent']
Subsidies_per_Category_short = Subsidies_per_Category.loc[:,['LSF subsidies', 'SSF subsidies','total_subsidies']]
print Subsidies_per_Category['SSF subsidies']/1000000
print Subsidies_per_Category['LSF subsidies']/1000000
Subsidies_per_Category['LSF percent'] = Subsidies_per_Category['LSF subsidies']/Subsidies_per_Category['total_subsidies']*100
print Subsidies_per_Category['total_subsidies']/1000000
Subsidies_per_Category_short = Subsidies_per_Category_short.reindex(index=['Beneficial','Capacity-enhancing','Ambiguous'])

Subsidies_per_Category_short['check'] = Subsidies_per_Category_short['LSF subsidies'] + Subsidies_per_Category_short['SSF subsidies']
np.sum(Subsidies_per_Category_short['SSF subsidies'])




In [ ]:

Total = Subsidies_per_Category_short.sum()
Total.name = 'Total'
Subsidies_per_Category_sum = Subsidies_per_Category_short.append(Total)
print Subsidies_per_Category_sum
print Subsidies_per_Category_sum['SSF subsidies']

print Subsidies_per_Category_sum['LSF subsidies']
print Subsidies_per_Category_sum['LSF subsidies'] / Subsidies_per_Category_sum['total_subsidies'] *100
print Subsidies_per_Category_sum['total_subsidies']

In [ ]:
#  Subsidies_per_Category_sum.append(Subsidies_per_Category_sum[2]/Subsidies_per_Category_sum[3])

In [ ]:


data = Subsidies_per_Category_short[['LSF subsidies','SSF subsidies']]
data_dict = data.to_dict()


# these percentages are the SSF percent out of the total subsidie calculated in Subsidies_per_Category['SSF percent']
SSFpercent = (
    {'label': 'Beneficial', 'percentage': 24.3}, 
    {'label': 'Capacity-enhancing', 'percentage': 10.6},
    {'label': 'Ambiguous', 'percentage': 14.8}
)

# add totals to SFFpercent
for key, val in data_dict.items():
    for k, v in data_dict[key].items():
        for i in SSFpercent:
            if i['label'] == k:
                if 'total' in i.keys():
                    i['total'] += v
                else:
                    i['total'] = v

# print("data_dict {}".format(data_dict))


# create subplot
ax = data.plot(kind='bar', figsize=(15,10),stacked=True, legend=True, fontsize=18, color=["darkgray","darkslategrey"])
ax.set_title("Global fisheries subsidies (2009)", fontsize=18)
ax.set_ylabel("USD x 10^10",fontsize=22)
ax.set_xlabel("",fontsize=18)
ax.set_xticklabels(['Beneficial','Capacity-enhancing','Ambiguous'],rotation=360, fontsize=18)
ax.set_axis_bgcolor('0.96')
ax.yaxis.labelpad = 20

# get an ordered list of percentages
percentages = []
for item in SSFpercent:
    for k, v in item.items():
        if k == 'percentage':
            percentages.append(v)

# prepare rects
N = len(SSFpercent)
indexes = np.arange(N)
width = 0.02
rects = ax.bar(indexes, percentages, width)

# iterate rects and map index with SSF percentage
for index, rect in enumerate(rects):
    SSFpercent[index]['rect_x'] = rect.get_x() 


# now we have a single object holding 
# all the values we need to label the chart 
for bar in SSFpercent:
    #  get percenatage label as a string
    percentage = bar['percentage']
    #  get value of to of bar
    height = bar['total']
    label_position = height - ((height / 100 * percentage) / 2)
    #  position text lables
    ax.text(
        bar['rect_x'] + width/2,
        label_position  - 12.4**5, str(percentage) + '%',
        ha="center",
        family="monospace",
        fontsize=16,
        fontweight="bold",
        color="white"
    )


plt.savefig('Subsidies_Category.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_Category.pdf', dpi=400, bbox_inches='tight')
plt.legend(loc=2,fontsize=18)
plt.show()

In [ ]:
SSF_subsidies_estimates_assessed.columns.values

In [ ]:
SSF_subsidies_estimates_assessed.replace(to_replace='Developed', value=1)
SSF = SSF_subsidies_estimates_assessed.loc[[330],['Developed']]
SSF

In [ ]:
Subsidies_per_subtype = SSF_subsidies_estimates_assessed.groupby(['SubType']).sum()
Subsidies_per_subtype = Subsidies_per_subtype.sort_values(by=['total_subsidies'],ascending=False)

Subsidies_per_subtype['SSF percent'] = Subsidies_per_subtype['SSF subsidies']/Subsidies_per_subtype['total_subsidies']*100
Subsidies_per_subtype_billions = Subsidies_per_subtype.loc[:,['LSF subsidies','SSF subsidies','SSF percent']]
Subsidies_per_subtype_billions['LSF subsidies'] = Subsidies_per_subtype_billions['LSF subsidies']/1000000
Subsidies_per_subtype_billions['SSF subsidies'] = Subsidies_per_subtype_billions['SSF subsidies']/1000000
print Subsidies_per_subtype_billions


In [ ]:
 Subsidies_per_subtype['total_subsidies']/35371111.003*100

In [ ]:
ax = Subsidies_per_subtype_billions[['LSF subsidies','SSF subsidies']].plot(kind='bar', figsize=(
        15,10),stacked=True, legend=True, fontsize=18, color=["darkgray","darkslategrey"] 
                                                                           )
ax.set_title("Global fisheries subsidies by Subtype (2009)", fontsize=18)
ax.set_ylabel("USD in billions",fontsize=18)
ax.yaxis.labelpad = 20
ax.set_xlabel("Subtype",fontsize=18)
ax.set_xticklabels(['Fuel', 'Management','Port Develop','Boat construct','Research and Develop','Market and storage','Vessel buyback','Fisher assistance','MPAs','Tax exemption','Develop project','Fishing access','Rural fisher communities'],rotation=80, fontsize=18)
ax.set_axis_bgcolor('0.96')
plt.legend(loc='best',fontsize=18)
plt.savefig('Subsidies_Subtype.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_Subtype.pdf', dpi=400, bbox_inches='tight')

plt.show()

In [ ]:
Subtypes = Subsidies_per_subtype.loc[:,['SSF percent', 'SSF subsidies','total_subsidies']]
Subtypes.to_csv('subtypes.csv')

In [ ]:
Developed_subtype = SSF_subsidies_estimates_assessed.groupby(['SubType','Developed']).sum()

Developed_short = Developed_subtype.loc[:,['SSF subsidies']]
Developed = Developed_short.reset_index()

Developed_pivot = Developed.pivot_table(['SSF subsidies'], index=['SubType'], columns='Developed')

Developed_grouped = Developed.groupby(['Developed','SubType']).sum()


Developed_stacked = Developed_grouped.stack()
Developed_stacked
Developed_pivot.to_csv('Developed_pivot.csv')
Developed_sorted = pd.read_csv('Developed_pivot_sorted.csv')
Developed_sorted = Developed_sorted.rename(columns={'Developing':'Developing countries', 'Developed':'Developed countries'})
print Developed_pivot
#  Developed_pivot['Total'] = Developed_pivot['False'] + Developed_pivot['True']
np.sum(Developed_pivot['SSF subsidies'])

In [ ]:
ax = Developed_sorted[['Developing countries','Developed countries']].plot(
    kind='bar', figsize=(14,8),stacked=True, legend=True, fontsize=20, color=["darkgray","darkslategrey"]
)
            
ax.set_title("Global small-scale fisheries subsidies", fontsize=23)
ax.set_titlepad = 100
ax.set_ylabel("USD in billion",fontsize=22)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=18)

ax.set_xticklabels([
        'Management','Port Develop','Research and Develop','Boat construct','Fisher assistance','Fuel','MPAs',
        'Develop project','Market and storage','Tax exemption','Rural fisher communities','Vessel buyback','Fishing access'
    ],rotation=80, fontsize=20
                  )
ax.set_axis_bgcolor('w')

legend = plt.legend(loc=5,fontsize=20)
legend.get_frame().set_facecolor('w')

plt.savefig('Subsidies_Developed.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_Developed.pdf', dpi=400, bbox_inches='tight')

plt.show()

In [ ]:
#  add another row to have a total value for developed and developing countries
#  calculate percentage for both developed and devleoping per subtype
Developed_sorted1 = Developed_sorted.append(Developed_sorted.sum(numeric_only=True), ignore_index=True)
Developed_sorted1['total'] = Developed_sorted1['Developing countries'] + Developed_sorted1['Developed countries']
Developed_sorted1['percent developed'] = Developed_sorted1['Developed countries'] / Developed_sorted1['total']*100
Developed_sorted1['percent developing'] = Developed_sorted1['Developing countries'] / Developed_sorted1['total']*100

Developed_sorted2 = Developed_sorted

In [ ]:
Developed_category = SSF_subsidies_estimates_assessed.groupby(['Category','Developed']).sum()

DevelopedC_short = Developed_category.loc[:,['SSF subsidies']]

print np.sum(DevelopedC_short['SSF subsidies'])

DevelopedC_short['SSF subsidies'] = DevelopedC_short['SSF subsidies']/1000000

DevelopedC = DevelopedC_short.unstack()
DevelopedC = DevelopedC.reindex(index=['Beneficial','Capacity-enhancing','Ambiguous'])
DevelopedC = DevelopedC.rename(columns={'SSF subsidies':'SSF'})
DevelopedC = DevelopedC.rename(columns={False:'Developing', True:'Developed'})

print DevelopedC


DevelopedC.to_csv('Developed_Category.csv')

#  DevelopedC_sorted = pd.read_csv('DevelopedC_pivot_sorted.csv')
#  DevelopedC_sorted = DevelopedC_sorted.rename(columns={'Developing':'Developing countries', 'Developed':'Developed countries'})
#  print Developed_pivot

Developed_sum = np.sum(DevelopedC['SSF'])

print Developed_sum

print

In [ ]:
DevelopedC_total = Developed_category.loc[:,['total_subsidies']]
print np.sum(DevelopedC_total['total_subsidies'])

DevelopedC_total['total_subsidies'] = DevelopedC_total['total_subsidies']/1000000

Developedtotal = DevelopedC_total.unstack()
Developedtotal = Developedtotal.reindex(index=['Beneficial','Capacity-enhancing','Ambiguous'])
Developedtotal = Developedtotal.rename(columns={'total_subsidies':'total'})
Developedtotal = Developedtotal.rename(columns={False:'Developing', True:'Developed'})


print Developedtotal
print np.sum(Developedtotal['total'])

In [ ]:
ax = DevelopedC.plot(
    kind='bar', figsize=(14,8),stacked=True, legend=[
        'Developed','Developing'], fontsize=20, color=["darkgray","darkslategrey"]
                    )
            
ax.set_title("Small-scale fisheries subsidies", fontsize=23)
ax.set_titlepad = 100
ax.set_ylabel("USD in billion",fontsize=22)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=18)

ax.set_xticklabels(['Beneficial','Capacity-enhancing','Ambiguous'],rotation=0, fontsize=20)
ax.set_axis_bgcolor('w')

legend = plt.legend(['Developing','Developed'],loc='best',fontsize=20)
legend.get_frame().set_facecolor('w')


plt.savefig('Subsidies_DevelopedC.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_DevelopedC.pdf', dpi=400, bbox_inches='tight')

plt.show()

In [ ]:
305.0/138.0

In [ ]:
SSF_subsidies_estimates_assessed.columns.values

In [ ]:
Subsidies_per_region = SSF_subsidies_estimates_assessed.groupby(['RegionName']).sum()
Subsidies_per_region['LSF subsidies'] = Subsidies_per_region['LSF subsidies']/1000000
Subsidies_per_region['SSF subsidies'] = Subsidies_per_region['SSF subsidies']/1000000
Subsidies_per_region = Subsidies_per_region.sort_values(by=['total_subsidies'],ascending=False)
Subsidies_per_region['SSF percent'] = (Subsidies_per_region['SSF subsidies']/(Subsidies_per_region['total_subsidies']/1000000)) *100
print Subsidies_per_region['total_subsidies']
print Subsidies_per_region['SSF percent'] 
print Subsidies_per_region['SSF subsidies']
print Subsidies_per_region['SSF subsidies']/np.sum(Subsidies_per_region['SSF subsidies'])*100
print Subsidies_per_region['LSF subsidies']/np.sum(Subsidies_per_region['LSF subsidies'])*100
print Subsidies_per_region['total_subsidies']/np.sum(Subsidies_per_region['total_subsidies'])*100

In [ ]:

data = Subsidies_per_region[['LSF subsidies','SSF subsidies']]
data_dict = data.to_dict()


# these percentages are the SSF percent out of the total subsidies calculated in Subsidies_per_region['SSF percent']
SSFpercent = (
    {'label': 'Asia', 'percentage': 18.6}, 
    {'label': 'Europe', 'percentage': 7},
    {'label': 'North America', 'percentage': 18.8},
    {'label': 'Oceania', 'percentage': 4.2},
    {'label': 'South, Central America and Carribbean', 'percentage': 30.9},
    {'label': 'Africa', 'percentage': 32.3}
)

# add totals to SFFpercent
for key, val in data_dict.items():
    for k, v in data_dict[key].items():
        for i in SSFpercent:
            #  print("i = {}, k = {}".format(i, k))
            if i['label'] == k:
                if 'total' in i.keys():
                    i['total'] += v
                else:
                    i['total'] = v

# make plot (also called subplot)
ax = data[['LSF subsidies','SSF subsidies']].plot(kind='bar', figsize=(15,10),stacked=True, legend=True, fontsize=18, color=["darkgray","darkslategrey"] )
ax.set_title("Global fisheries subsidies by Region (2009)", fontsize=18)
ax.set_ylabel("USD in milions",fontsize=18)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=18)
ax.set_xticklabels(['Asia', 'Europe','N America','Oceania','S + C America,Carribbean', 'Africa'],rotation=80, fontsize=18)
ax.set_axis_bgcolor('0.96')
plt.legend(loc='best',fontsize=18)


# get an ordered list of percentages
percentages = []
for item in SSFpercent:
    for k, v in item.items():
        if k == 'percentage':
            percentages.append(v)
            
# prepare rects (rectangles)
N = len(SSFpercent)
indexes = np.arange(N)
width = 0.02
rects = ax.bar(indexes, percentages, width)

# iterate rects and map index with SSF percentage
for index, rect in enumerate(rects):
    SSFpercent[index]['rect_x'] = rect.get_x() 

    # now we have a single object holding 
# all the values we need to label the chart 
for bar in SSFpercent:
    #print bar
    #  get percentage label as a string
    percentage = bar['percentage']
    #  get value of to of bar
    height = bar['total']
    label_position = height - ((height / 100 * percentage))
    #  position text lables
    ax.text(
        bar['rect_x'] + width/2,
        label_position  + 1**5, str(percentage) + '%',
        ha="center",
        family="monospace",
        fontsize=18,
        fontweight="bold",
        color="black"
    )

    


plt.show()

In [ ]:
ax = data[['LSF subsidies','SSF subsidies']].plot(kind='bar', figsize=(15,10),stacked=True, legend=True, fontsize=18, color=["darkgray","darkslategrey"] )
ax.set_title("Global fisheries subsidies by Region (2009)", fontsize=18)
ax.set_ylabel("USD in milions",fontsize=18)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=18)
ax.set_xticklabels(['Asia', 'Europe','N America','Oceania','S + C America,Carribbean', 'Africa'],rotation=80, fontsize=18)
ax.set_axis_bgcolor('0.96')
plt.legend(loc='best',fontsize=18)

plt.savefig('Subsidies_Region.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_Region.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
Subsidies_per_developed = SSF_subsidies_estimates_assessed.groupby(['Developed']).sum()
Subsidies_per_developed

In [ ]:
Subsidies_per_developed['SSF_percent'] = Subsidies_per_developed['SSF subsidies']/Subsidies_per_developed['total_subsidies']*100
Subsidies_per_developed['SSF_percent']

In [ ]:
Subsidies_developed_category = SSF_subsidies_estimates_assessed.groupby(['Developed','Category']).sum()
Subsidies_developed_category['SSF_percent'] = Subsidies_developed_category['SSF subsidies']/Subsidies_developed_category['total_subsidies']*100
Subsidies_developed_category['SSF_percent']

In [ ]:
Subsidies_per_Subregion = SSF_subsidies_estimates_assessed.groupby(['Subregion']).sum()
#  print Subsidies_per_Subregion.loc[:,['total_subsidies']]

In [ ]:
Subregions_all = Subsidies_per_Subregion.loc[:,['total_subsidies', 'SSF subsidies','LSF subsidies']]

Subregions_all_transposed = Subsidies_per_Subregion.loc[:,['SSF subsidies','LSF subsidies']].transpose()
Subregions_all_transposed

In [ ]:
#  Subsidies_per_Subregion['total_size'] = Subsidies_per_Subregion['total_subsidies']/100000
#  Subsidies_per_Subregion['total_size'] 
Subregions_all.to_csv('Subsidies_per_Subregion.csv')

In [ ]:
#  Europe = SSF_subsidies_estimates_assessed.pivot_table(['total_subsidies','SSF subsidies','LSF subsidies'], index=None, columns='EU member')
Europe = SSF_subsidies_estimates_assessed.groupby(['EU member']).sum()
Europe = Europe.loc[[1],['LSF subsidies','SSF subsidies','total_subsidies']]
Europe['LSF subsidies'] = Europe['LSF subsidies']/1000000
Europe['SSF subsidies'] = Europe['SSF subsidies']/1000000
Europe['total_subsidies'] = Europe['total_subsidies']/1000000
Europe = Europe.rename(index={True:'EU'})

In [ ]:
Europe

In [ ]:
#  sorting and grouping data by largest fishing nations and political entities to prepare for a graph

Subsidies_countries = SSF_subsidies_estimates_assessed.groupby(['Countries']).sum()
print Subsidies_countries.columns.values

#  catch.groupby()
Subsidies_countries_fishing = Subsidies_countries.loc[['Japan','China','USA','Russian Fed','Indonesia'],['LSF subsidies','SSF subsidies','total_subsidies']]

Subsidies_countries_fishing['LSF subsidies'] = Subsidies_countries_fishing['LSF subsidies']/1000000
Subsidies_countries_fishing['SSF subsidies'] = Subsidies_countries_fishing['SSF subsidies']/1000000
Subsidies_countries_fishing['total_subsidies'] = Subsidies_countries_fishing['total_subsidies']/1000000

Subsidies_fishing = pd.concat([Subsidies_countries_fishing, Europe])


Subsidies_fishing = Subsidies_fishing.sort_values(by=['total_subsidies'], ascending = False)

print Subsidies_fishing

print np.sum(SSF_subsidies_estimates_assessed['LSF subsidies'])/1000000
print np.sum(SSF_subsidies_estimates_assessed['total_subsidies'])/1000000
print np.sum(SSF_subsidies_estimates_assessed['SSF subsidies'])/1000000


#  Subsidies_fishing.reindex_set[['Japan','China','USA','Russia','Indonesia','EU']]
#  Subsidies_countries = Subsidies_countries.sort_values(by=['total_subsidies'], ascending=False)

In [ ]:
Subsidies_fishing
ax = Subsidies_fishing[['LSF subsidies','SSF subsidies']].plot(kind='bar', figsize=(15,10),stacked=True, legend=True, fontsize=20, color=["darkgray","darkslategrey"] )
ax.set_title("Global fisheries subsidies by major fishing nations/political entities (2009)", fontsize=20)
ax.set_ylabel("USD in billions",fontsize=20)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=20)
ax.set_xticklabels(['EU', 'Japan', 'China','USA','Russia', 'Indonesia'],rotation=0, fontsize=18)
ax.set_axis_bgcolor('w')

legend = plt.legend(loc='best',fontsize=20)
legend.get_frame().set_facecolor('w')


plt.savefig('Subsidies_fishing.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_fishing.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
print Subsidies_fishing

print np.sum(SSF_subsidies_estimates_assessed['LSF subsidies'])/1000000
totalsubsidies = np.sum(SSF_subsidies_estimates_assessed['total_subsidies'])/1000000
print np.sum(SSF_subsidies_estimates_assessed['SSF subsidies'])/1000000

Percent_of_totalSubsidies = (Subsidies_fishing['LSF subsidies'] + Subsidies_fishing['SSF subsidies'])/totalsubsidies*100
SSFPercent = (Subsidies_fishing['SSF subsidies'] / Subsidies_fishing['total_subsidies'])*100
#  print Percent_of_totalSubsidies
#  print SSFPercent

In [ ]:
#  sorting and grouping data by biggest subsidysing nations to graph them

Subsidies_countries_major = Subsidies_countries.loc[['Japan','China','USA','Russian Fed','Micronesia','Spain',
    'Korea Rep','Canada','Philippines'],[
    'total_subsidies','SSF subsidies','LSF subsidies']
                                                   ]
#  to present values in USD billions
Subsidies_countries_major['LSF subsidies'] = Subsidies_countries_major['LSF subsidies']/1000000
Subsidies_countries_major['SSF subsidies'] = Subsidies_countries_major['SSF subsidies']/1000000
Subsidies_countries_major['total_subsidies'] = Subsidies_countries_major['total_subsidies']/1000000

#  include EU into table
Subsidies_major = pd.concat([Subsidies_countries_major, Europe])

#  sorting values by largest subsidies
Subsidies_major = Subsidies_major.sort_values(by=['total_subsidies'], ascending=False)

totalSSFsubsidies = (np.sum(SSF_subsidies_estimates_assessed['SSF subsidies']))/1000000

Subsidies_major_percent = (np.sum(Subsidies_major['total_subsidies'])/totalsubsidies)*100
Subsidies_major_SSFpercent = (np.sum(Subsidies_major['SSF subsidies'])/totalSSFsubsidies)*100
print np.sum(Subsidies_major['total_subsidies'])
print totalsubsidies
print Subsidies_major_percent
print Subsidies_major_SSFpercent
Subsidies_major

In [ ]:
Subsidies_major
ax = Subsidies_major[['LSF subsidies','SSF subsidies']].plot(kind='bar', figsize=(15,10),stacked=True, legend=True, fontsize=20, color=["darkgray","darkslategrey"] )
ax.set_title("Global fisheries subsidies major subsidyzing nations and the EU", fontsize=20)
ax.set_ylabel("USD in billions",fontsize=20)
ax.yaxis.labelpad = 20
ax.set_xlabel("",fontsize=20)
ax.set_xticklabels(['EU','Japan','China','USA','Russia','Micronesia','Spain','Korea Rep','Canada','Philippines'],rotation=80, fontsize=20)

ax.set_axis_bgcolor('w')

legend = plt.legend(loc=5,fontsize=20)
legend.get_frame().set_facecolor('w')

plt.savefig('Subsidies_major.svg', dpi=400, bbox_inches='tight')
plt.savefig('Subsidies_major.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
Subsidies_countries_major['SSF percent'] = Subsidies_countries_major['SSF subsidies']/Subsidies_countries_major['total_subsidies']*100 

In [ ]:
Subsidies_countries_major.transpose()

In [ ]:
#  work with SSF_subsidies
#  organize data by data group which is 1,2 or 3 and have frequencies per subytype of all assessed subsidies data
#  this includes 73 countries that have been assessed
print SSF_subsidies.columns.values
Data_group = pd.crosstab(SSF_subsidies.SubType, SSF_subsidies.Data_group)
print Data_group.columns.values
Data_group = Data_group.reset_index()
print Data_group
#  making a new index for the columns to work with the new dataframe
#  Data_group.set_index(['ID','SubType','one','two','three'], verify_integrity=False)

In [ ]:
Data_group_percentages = pd.crosstab(SSF_subsidies.SubType, SSF_subsidies.Data_group).apply(lambda r: r/r.sum(), axis=1)
Data_group_percentages

In [ ]:
ax = Data_group_percentages.plot(kind='bar', figsize=(15,10), stacked = True, fontsize=18, color=["black","dimgray", "darkgray"])
ax.set_title("Data groups of assessed small-scale fisheries subsidy information", fontsize=18)
ax.set_ylabel("Frequencies",fontsize=18)
ax.yaxis.labelpad = 20
ax.set_axis_bgcolor('w')
plt.legend(loc=1,fontsize=18)

plt.savefig('Data_groups_percentages.svg', dpi=400, bbox_inches='tight')
plt.savefig('Data_groups_percentages.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
ax = Data_group.plot(kind='bar', figsize=(15,10), stacked = True, fontsize=18, color=["black","dimgray", "darkgray"])
ax.set_title("Data groupd of assessed small-scale fisheries subsidy information", fontsize=18)
ax.set_ylabel("Frequencies",fontsize=18)
ax.yaxis.labelpad = 20
ax.set_axis_bgcolor('w')
plt.legend(loc='best',fontsize=18)

plt.savefig('Data_groups.svg', dpi=400, bbox_inches='tight')
plt.savefig('Data_groups.pdf', dpi=400, bbox_inches='tight')
plt.show()



In [ ]:

Subsidies_Data_Table = SSF_subsidies_estimates_assessed.groupby(['Countries','SubType']).sum()
print Subsidies_Data_Table.columns.values
Appendix = Subsidies_Data_Table.loc[:,['SSF subsidies','LSF subsidies','Data_group','SSFsubsidies_assessed_percent']]

In [ ]:
Appendix2_fuel = pd.read_csv("SSF_subsidies_appendix_fuel.csv")

In [ ]:
Appendix2 = pd.merge(left=all_subsidies, right=Appendix2_fuel, how='left')


In [ ]:
SSF_subsidies_estimates_assessed.to_csv('Appendix2.csv')